In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from keras.layers import *
from keras import optimizers
from keras import backend as K
from sklearn.metrics import f1_score
# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')
# load feats
all = True
if all:
    train_x,test_x = [],[]
    for feat in sorted(glob.glob('../feature/stacking_pkl_file/*')):
        if 'fm2.pkl' in feat or 'ligbm' in  feat :
            continue
        print('file path',feat)
        a,b = pickle.load(open(feat,'rb'))
        print(a.shape,b.shape)
        train_x.append(a)
        test_x.append(b)
else:
    train_x,test_x = [],[]
    for feat in sorted(glob.glob('../feature/staclomg_pkl_file_nn/*')):
        if 'fm2.pkl' in feat or 'ligbm' in feat :
            continue
        print('file path',feat)
        a,b = pickle.load(open(feat,'rb'))
        print(a.shape,b.shape)
        train_x.append(a)
        test_x.append(b)

    

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


file path ../feature/stacking_pkl_file/Capsule.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_all_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/pseudo_rnn2.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn7879.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_all_shuffle_all.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_pseudo1.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_shuffle1.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_f

In [2]:
for feat in sorted(glob.glob('../feature/other_features/features-vinson/*')):
    if 'fm2.pkl' in feat or 'w2v' in feat or 'lda' in feat or 'lsi' in feat:
        continue
    print('file path',feat)
    file=pd.read_csv(feat)
    a = file.iloc[:102277,:]
    b = file.iloc[102277:,:]
    a = a.values
    b = b.values
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)

file path ../feature/other_features/features-vinson/bnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob_0.778620.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/mnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/nn_ensemble_0.775597.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob_0.778881.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/word_fasttext_0.760225.csv
(102277, 19) (102277, 19)
(102277, 399)


In [3]:
train = pd.read_csv("../input/new_data/train_set.csv")

In [4]:
y=(train["class"]-1).astype(int)
from keras.utils import np_utils
train_y=np_utils.to_categorical(y,num_classes=20)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [5]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [6]:
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
def get_nn():
    cv1_input = Input(shape=(399,), dtype='float32')
#     merged = concatenate([cv1_input])
    merged = Dense(512,activation='relu')(cv1_input)
    merged = Dropout(0.2)(merged)
    x = Dense(20, activation="softmax")(merged)
    model = Model(inputs=cv1_input, outputs=x)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                    metrics=[f1])
    return model
print('def model done')

def model done


In [7]:
from sklearn.model_selection import KFold
import gc
def kf_train(fold_cnt=3,rnd=1):
    now_nfold=0
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    test_pred, train_pred = np.zeros((102277,20)),np.zeros((102277,20))
#     LRDecay = LearningRateScheduler(lr_decay)
    for train_index, test_index in kf.split(train_x):
        
        now_nfold+=1
        print ("now is {} fold".format(now_nfold))
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        d_test = test_x
        batch_size = 64
        epochs = 15

        model = get_nn()

        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, save_best_only=True,verbose=1, monitor='val_f1',  mode='max')
        early = EarlyStopping( monitor='val_f1',  mode='max', patience=2,)
        callbacks_list = [checkpoint,early]

        model.fit(curr_x, curr_y, 
                  batch_size=batch_size, epochs=epochs, 
                    validation_data=(hold_out_x, hold_out_y), 
                  callbacks=callbacks_list)

        
        model.load_weights(file_path)
       
        train_pred[test_index] = model.predict(hold_out_x)
        curr_test_pred = model.predict(d_test)
        test_pred += curr_test_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    return train_pred, test_pred


print('def done')

def done


In [8]:
train_pred, nn_res = kf_train(fold_cnt=10,rnd=4)
#### first fold #######8141 0.796<lb<0.797 

now is 1 fold
Train on 92049 samples, validate on 10228 samples
Epoch 1/15
92049/92049 [==============================] - 14s 153us/step - loss: 0.7196 - f1: 0.8062 - val_loss: 0.6805 - val_f1: 0.8154

Epoch 00001: val_f1 improved from -inf to 0.81541, saving model to weights_base.best.h5
Epoch 2/15
92049/92049 [==============================] - 7s 78us/step - loss: 0.6593 - f1: 0.8157 - val_loss: 0.6578 - val_f1: 0.8161

Epoch 00002: val_f1 improved from 0.81541 to 0.81610, saving model to weights_base.best.h5
Epoch 3/15
92049/92049 [==============================] - 7s 78us/step - loss: 0.6410 - f1: 0.8169 - val_loss: 0.6474 - val_f1: 0.8166

Epoch 00003: val_f1 improved from 0.81610 to 0.81664, saving model to weights_base.best.h5
Epoch 4/15
92049/92049 [==============================] - 7s 78us/step - loss: 0.6323 - f1: 0.8187 - val_loss: 0.6487 - val_f1: 0.8138

Epoch 00004: val_f1 did not improve from 0.81664
Epoch 5/15
92049/92049 [==============================] - 7s 75us/step 

92050/92050 [==============================] - 7s 77us/step - loss: 0.6330 - f1: 0.8181 - val_loss: 0.6366 - val_f1: 0.8191

Epoch 00004: val_f1 improved from 0.81847 to 0.81907, saving model to weights_base.best.h5
Epoch 5/15
92050/92050 [==============================] - 7s 76us/step - loss: 0.6264 - f1: 0.8182 - val_loss: 0.6366 - val_f1: 0.8215

Epoch 00005: val_f1 improved from 0.81907 to 0.82148, saving model to weights_base.best.h5
Epoch 6/15
92050/92050 [==============================] - 7s 73us/step - loss: 0.6200 - f1: 0.8195 - val_loss: 0.6268 - val_f1: 0.8209

Epoch 00006: val_f1 did not improve from 0.82148
Epoch 7/15
92050/92050 [==============================] - 7s 71us/step - loss: 0.6155 - f1: 0.8194 - val_loss: 0.6306 - val_f1: 0.8213

Epoch 00007: val_f1 did not improve from 0.82148
now is 9 fold
Train on 92050 samples, validate on 10227 samples
Epoch 1/15
92050/92050 [==============================] - 7s 73us/step - loss: 0.7134 - f1: 0.8080 - val_loss: 0.7135 - val

In [9]:
with open('../feature/2leve_nn_stacking2.pkl','wb') as fout:
    pickle.dump([train_pred,nn_res],fout)

In [10]:
test_dir = '../input/new_data/test_set.csv'
test = pd.read_csv(test_dir)
test_id = test[["id"]].copy()

In [11]:
preds=np.argmax(nn_res,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["class"]
test_pred["class"]=(test_pred["class"]+1).astype(int)
print(test_pred.shape)
print(test_id.shape)
test_pred["id"]=list(test_id["id"])
test_pred[["id","class"]].to_csv('../output/nnstack5.csv',index=None)

(102277, 1)
(102277, 1)


In [12]:
name = ["class_prob_%s"%i for i in range(1,21)]
df_nn = pd.DataFrame(nn_res, columns=name)
df_nn = df_nn.drop('class_prob_20', axis=1)
df_nn.to_csv('../pro/stacking_nn5.csv', index=False)

In [13]:
nn_res.shape

(102277, 20)

In [14]:
df_nn

,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,class_prob_10,class_prob_11,class_prob_12,class_prob_13,class_prob_14,class_prob_15,class_prob_16,class_prob_17,class_prob_18,class_prob_19
0,0.002993,0.000245,0.001125,0.000995,0.985771,0.000752,0.000198,0.000876,0.000093,0.001269,0.000546,0.001497,0.000791,0.000891,0.000256,0.000327,0.000184,0.000359,0.000833
1,0.004403,0.000646,0.000334,0.984887,0.000505,0.000564,0.001873,0.000194,0.000068,0.000761,0.000535,0.001523,0.000781,0.000965,0.000206,0.000159,0.000131,0.000597,0.000868
2,0.020216,0.003476,0.020515,0.003525,0.006963,0.004055,0.005920,0.066945,0.004388,0.005421,0.012484,0.032441,0.780038,0.003759,0.005350,0.000229,0.003238,0.003258,0.017775
3,0.000698,0.000111,0.000133,0.996653,0.000171,0.000301,0.000465,0.000091,0.000012,0.000174,0.000136,0.000225,0.000198,0.000262,0.000052,0.000030,0.000051,0.000049,0.000187
4,0.012177,0.000401,0.001422,0.001217,0.963366,0.001113,0.001120,0.001980,0.000238,0.005025,0.001445,0.001602,0.002569,0.002457,0.000358,0.000348,0.000299,0.000540,0.002322
5,0.022627,0.001485,0.005433,0.002175,0.919910,0.003880,0.000598,0.007751,0.000745,0.013252,0.004102,0.005181,0.003891,0.001761,0.000876,0.001060,0.000546,0.002546,0.002178
6,0.005423,0.000255,0.003822,0.000366,0.000080,0.001282,0.000477,0.001258,0.000284,0.005662,0.001915,0.002070,0.000461,0.001747,0.967127,0.000248,0.001203,0.000921,0.005398
7,0.035851,0.157296,0.008682,0.005366,0.006395,0.008245,0.063860,0.019170,0.008383,0.017275,0.009025,0.018115,0.020013,0.008736,0.011118,0.001392,0.066895,0.025969,0.508186
8,0.000337,0.000018,0.994240,0.000084,0.000059,0.001662,0.000078,0.000039,0.000065,0.000176,0.000367,0.001420,0.000250,0.000192,0.000417,0.000045,0.000024,0.000132,0.000395
9,0.087528,0.004125,0.003935,0.005921,0.005538,0.005220,0.020393,0.177747,0.018016,0.073854,0.013229,0.473774,0.020344,0.003572,0.002183,0.005326,0.002417,0.020929,0.055936


In [10]:
nn_res[0]

array([4.70125763e-03, 9.41594957e-04, 4.31315157e-04, 1.03147708e-03,
       9.80580300e-01, 6.71539639e-04, 4.68012059e-04, 2.37537872e-03,
       1.03082371e-04, 1.38701548e-03, 9.77989039e-04, 2.48241170e-03,
       4.65297431e-04, 5.17155064e-04, 3.12560847e-04, 1.24375290e-03,
       1.98021449e-04, 3.04372875e-04, 8.06763067e-04, 7.34517270e-07])